In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv
/kaggle/input/world-population-19602018/population_total_long.csv
/kaggle/input/world-population-19602018/population_above_age_65_percentage_long.csv
/kaggle/input/world-population-19602018/population_female_percentage_long.csv
/kaggle/input/world-population-19602018/population_density_long.csv
/kaggle/input/world-population-19602018/population_below_age_14_percentage_long.csv
/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200206.csv
/kaggle/input/2019-coronavirus-dataset-01212020-01262020/2019_nCoV_20200121_20200

In [36]:
#Importing required libraries
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [37]:
from datetime import timedelta
import calmap
import folium


In [38]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [39]:
import warnings
warnings.filterwarnings('ignore')


In [40]:
from IPython.display import Javascript
from IPython.core.display import display
from IPython.core.display import HTML

In [41]:
#defining the color codings
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801'


In [42]:
#Importing Datset into a DataFrame
full_table = pd.read_csv("/kaggle/input/corona-virus-report/covid_19_clean_complete.csv", parse_dates=['Date'])
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0


In [43]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16368 entries, 0 to 16367
Data columns (total 8 columns):
Province/State    4950 non-null object
Country/Region    16368 non-null object
Lat               16368 non-null float64
Long              16368 non-null float64
Date              16368 non-null datetime64[ns]
Confirmed         16368 non-null int64
Deaths            16368 non-null int64
Recovered         16368 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1023.1+ KB


In [44]:
#checking for missing value,
full_table.isna().sum()

Province/State    11418
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [45]:
cases = ['Confirmed', 'Deaths', 'Recovered', 'Active']

full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


In [46]:
full_table.head(4)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0


In [47]:
#filling missing values in 'Provinvce/State' column
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table[cases] = full_table[cases].fillna(0)

In [49]:
full_table.head(4)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0


In [64]:
#Cases in the ships (Diamond Princess | Grand Princess)
ship =  full_table[full_table['Province/State'].str.contains('Grand Princess')| full_table['Country/Region'].str.contains('Diamond Princess')]
ship

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
37,Grand Princess,Canada,37.6489,-122.6655,2020-01-22,0,0,0,0
88,,Diamond Princess,0.0000,0.0000,2020-01-22,0,0,0,0
285,Grand Princess,Canada,37.6489,-122.6655,2020-01-23,0,0,0,0
336,,Diamond Princess,0.0000,0.0000,2020-01-23,0,0,0,0
533,Grand Princess,Canada,37.6489,-122.6655,2020-01-24,0,0,0,0
...,...,...,...,...,...,...,...,...,...
15712,,Diamond Princess,0.0000,0.0000,2020-03-25,712,10,587,115
15909,Grand Princess,Canada,37.6489,-122.6655,2020-03-26,13,0,0,13
15960,,Diamond Princess,0.0000,0.0000,2020-03-26,712,10,597,105
16157,Grand Princess,Canada,37.6489,-122.6655,2020-03-27,13,0,0,13


In [65]:
#China and rest of world
china = full_table[full_table['Country/Region']== 'China']
row = full_table[full_table['Country/Region'] != 'China']


In [67]:
#latest records
full_latest = full_table[full_table['Date'] == max(full_table['Date'])].reset_index()
full_latest

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,16120,,Afghanistan,33.000000,65.000000,2020-03-27,110,4,2,104
1,16121,,Albania,41.153300,20.168300,2020-03-27,186,8,31,147
2,16122,,Algeria,28.033900,1.659600,2020-03-27,409,26,29,354
3,16123,,Andorra,42.506300,1.521800,2020-03-27,267,3,1,263
4,16124,,Angola,-11.202700,17.873900,2020-03-27,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...
243,16363,,Saint Kitts and Nevis,17.357822,-62.782998,2020-03-27,2,0,0,2
244,16364,Northwest Territories,Canada,64.825500,-124.845700,2020-03-27,1,0,0,1
245,16365,Yukon,Canada,64.282300,-135.000000,2020-03-27,3,0,0,3
246,16366,,Kosovo,42.602636,20.902977,2020-03-27,86,1,1,84


In [68]:
china_latest = china[china['Date'] == max(china['Date'])].reset_index()
china_latest

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,16169,Anhui,China,31.8257,117.2264,2020-03-27,990,6,984,0
1,16170,Beijing,China,40.1824,116.4142,2020-03-27,569,8,408,153
2,16171,Chongqing,China,30.0572,107.8740,2020-03-27,578,6,570,2
3,16172,Fujian,China,26.0789,117.9874,2020-03-27,331,1,295,35
4,16173,Gansu,China,37.8099,101.0583,2020-03-27,136,2,123,11
5,16174,Guangdong,China,23.3417,113.4244,2020-03-27,1456,8,1337,111
6,16175,Guangxi,China,23.8298,108.7881,2020-03-27,254,2,250,2
7,16176,Guizhou,China,26.8154,106.8748,2020-03-27,146,2,144,0
8,16177,Hainan,China,19.1959,109.7453,2020-03-27,168,6,168,-6
9,16178,Hebei,China,39.5490,116.1306,2020-03-27,319,6,310,3


In [72]:
row_latest = row[row['Date'] == max(row['Date'])].reset_index()
row_latest

,index,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,16120,,Afghanistan,33.000000,65.000000,2020-03-27,110,4,2,104
1,16121,,Albania,41.153300,20.168300,2020-03-27,186,8,31,147
2,16122,,Algeria,28.033900,1.659600,2020-03-27,409,26,29,354
3,16123,,Andorra,42.506300,1.521800,2020-03-27,267,3,1,263
4,16124,,Angola,-11.202700,17.873900,2020-03-27,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...
210,16363,,Saint Kitts and Nevis,17.357822,-62.782998,2020-03-27,2,0,0,2
211,16364,Northwest Territories,Canada,64.825500,-124.845700,2020-03-27,1,0,0,1
212,16365,Yukon,Canada,64.282300,-135.000000,2020-03-27,3,0,0,3
213,16366,,Kosovo,42.602636,20.902977,2020-03-27,86,1,1,84


In [78]:
full_latest_grouped = full_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Active', 'Recovered'].sum().reset_index()

In [80]:
china_latest_grouped = china_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
china_latest_grouped

,Country/Region,Confirmed,Deaths,Recovered,Active
0,China,81897,3296,74720,3881


In [82]:
row_latest_grouped = row_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
row_latest_grouped

,Country/Region,Confirmed,Deaths,Recovered,Active
0,Afghanistan,110,4,2,104
1,Albania,186,8,31,147
2,Algeria,409,26,29,354
3,Andorra,267,3,1,263
4,Angola,4,0,0,4
...,...,...,...,...,...
170,Venezuela,107,1,31,75
171,Vietnam,163,0,20,143
172,West Bank and Gaza,91,1,17,73
173,Zambia,22,0,0,22


In [84]:
temp = full_table.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered', 'Active'].max()
temp

,,Confirmed,Deaths,Recovered,Active
Country/Region,Province/State,,,,
Afghanistan,,110,4,2,104
Albania,,186,8,31,151
Algeria,,409,26,65,354
Andorra,,267,3,1,263
Angola,,4,0,0,4
...,...,...,...,...,...
Venezuela,,107,1,31,92
Vietnam,,163,0,20,143
West Bank and Gaza,,91,1,17,73


In [85]:
temp = full_table.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp

,Date,Confirmed,Deaths,Recovered,Active
0,2020-01-22,555,17,28,510
1,2020-01-23,654,18,30,606
2,2020-01-24,941,26,36,879
3,2020-01-25,1434,42,39,1353
4,2020-01-26,2118,56,52,2010
...,...,...,...,...,...
61,2020-03-23,378235,16505,98341,263389
62,2020-03-24,418045,18625,107890,291530
63,2020-03-25,467653,21181,113604,332868
64,2020-03-26,529591,23970,121966,383655


temp = temp[temp['Date'] == max(temp['Date'])].reset_index(drop=True)
temp

In [92]:
temp = temp[temp['Date'] == max(temp['Date'])].reset_index(drop=True)
temp.style.background_gradient(cmap='inferno_r')

,Date,Confirmed,Deaths,Recovered,Active,Global Mortality,Deaths per 100 Confirmed Cases
0,2020-03-27 00:00:00,593291,27198,130659,435434,0.0458426,4.58426


In [93]:
temp['Global Mortality'] = temp['Deaths']/temp['Confirmed']
temp['Deaths per 100 Confirmed Cases'] = temp['Global Mortality']*100
temp.style.background_gradient(cmap='Pastel1')


,Date,Confirmed,Deaths,Recovered,Active,Global Mortality,Deaths per 100 Confirmed Cases
0,2020-03-27 00:00:00,593291,27198,130659,435434,0.0458426,4.58426


In [94]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)



In [96]:
tm = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path=['variable'], values="value", height=400, width=600, color_discrete_sequence=[act, rec, dth])
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [98]:
temp_f = full_latest_grouped.sort_values(by='Confirmed', ascending=False)
temp_f

,Country/Region,Confirmed,Deaths,Active,Recovered
164,US,101657,1581,99207,869
82,Italy,86498,9134,66414,10950
34,China,81897,3296,3881,74720
149,Spain,65719,5138,51224,9357
63,Germany,50871,342,43871,6658
...,...,...,...,...,...
69,Guinea-Bissau,2,0,2,0
125,Papua New Guinea,1,0,1,0
137,Saint Vincent and the Grenadines,1,0,1,0
159,Timor-Leste,1,0,1,0


In [99]:
temp_f = temp_f[['Country/Region', 'Confirmed', 'Active',  'Deaths', 'Recovered']]
temp_f = temp_f.reset_index(drop=True)
temp_f

,Country/Region,Confirmed,Active,Deaths,Recovered
0,US,101657,99207,1581,869
1,Italy,86498,66414,9134,10950
2,China,81897,3881,3296,74720
3,Spain,65719,51224,5138,9357
4,Germany,50871,43871,342,6658
...,...,...,...,...,...
171,Guinea-Bissau,2,2,0,0
172,Papua New Guinea,1,1,0,0
173,Saint Vincent and the Grenadines,1,1,0,0
174,Timor-Leste,1,1,0,0


In [101]:
temp_f.style.background_gradient(cmap='Blues', subset=['Confirmed', 'Active'])

,Country/Region,Confirmed,Active,Deaths,Recovered
0,US,101657,99207,1581,869
1,Italy,86498,66414,9134,10950
2,China,81897,3881,3296,74720
3,Spain,65719,51224,5138,9357
4,Germany,50871,43871,342,6658
5,France,33402,25698,1997,5707
6,Iran,32332,18821,2378,11133
7,United Kingdom,14745,13833,761,151
8,Switzerland,12928,11167,231,1530
9,South Korea,9332,4665,139,4528


In [102]:
temp_f.style.background_gradient(cmap="Blues", subset=['Confirmed', 'Active'])\
            .background_gradient(cmap="Greens", subset=['Recovered'])\
            .background_gradient(cmap="Reds", subset=['Deaths'])

,Country/Region,Confirmed,Active,Deaths,Recovered
0,US,101657,99207,1581,869
1,Italy,86498,66414,9134,10950
2,China,81897,3881,3296,74720
3,Spain,65719,51224,5138,9357
4,Germany,50871,43871,342,6658
5,France,33402,25698,1997,5707
6,Iran,32332,18821,2378,11133
7,United Kingdom,14745,13833,761,151
8,Switzerland,12928,11167,231,1530
9,South Korea,9332,4665,139,4528


In [103]:
temp_flg = temp_f[temp_f['Deaths']>0][['Country/Region', 'Deaths']]
temp_flg['Deaths / 100 Cases'] = round((temp_f['Deaths']/temp_f['Confirmed'])*100)
temp_flg

,Country/Region,Deaths,Deaths / 100 Cases
0,US,1581,2.0
1,Italy,9134,11.0
2,China,3296,4.0
3,Spain,5138,8.0
4,Germany,342,1.0
...,...,...,...
152,Guyana,1,20.0
153,Zimbabwe,1,20.0
162,Sudan,1,33.0
164,Gambia,1,33.0


In [104]:
temp_flg.sort_values('Deaths', ascending=False).reset_index(drop=True).style.background_gradient(cmap='Reds')


,Country/Region,Deaths,Deaths / 100 Cases
0,Italy,9134,11
1,Spain,5138,8
2,China,3296,4
3,Iran,2378,7
4,France,1997,6
5,US,1581,2
6,United Kingdom,761,5
7,Netherlands,547,6
8,Germany,342,1
9,Belgium,289,4


In [107]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(full_latest)):
    folium.Circle(
        location=[full_latest.iloc[i]['Lat'], full_latest.iloc[i]['Long']],
        color='crimson', 
        tooltip =   '<li><bold>Country : '+str(full_latest.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(full_latest.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(full_latest.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(full_latest.iloc[i]['Deaths']),
        radius=int(full_latest.iloc[i]['Confirmed'])**1.1).add_to(m)
m

In [111]:
# Deaths
temp = full_latest_grouped[full_latest_grouped['Deaths']>0]
fig = px.choropleth(temp, 
                    locations="Country/Region", locationmode='country names',
                    color=np.log(temp["Deaths"]), hover_name="Country/Region", 
                    color_continuous_scale="Peach", hover_data=['Deaths'],
                    title='Countries with Deaths Reported')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [112]:
formated_gdf = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')
formated_gdf['size'] = formated_gdf['Confirmed'].pow(0.3)

fig = px.scatter_geo(formated_gdf, locations="Country/Region", locationmode='country names', 
                     color="Confirmed", size='size', hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf['Confirmed'])+2], animation_frame="Date", 
                     title='Spread over time')
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [113]:
# ------------------------------------------------------------------------------------------

temp = full_table.groupby('Date')['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()
temp['No. of Deaths to 100 Confirmed Cases'] = round(temp['Deaths']/temp['Confirmed'], 3)*100
temp['No. of Recovered to 100 Confirmed Cases'] = round(temp['Recovered']/temp['Confirmed'], 3)*100

fig_1 = px.area(temp, x="Date", y="Confirmed", color_discrete_sequence = [act])
fig_2 = px.area(temp, x="Date", y="Deaths", color_discrete_sequence = [dth])
fig_3 = px.line(temp, x="Date", y="No. of Deaths to 100 Confirmed Cases",  color_discrete_sequence=['#333333'])
fig_4 = px.line(temp, x="Date", y="No. of Recovered to 100 Confirmed Cases",  color_discrete_sequence=['#0f4c75'])

# ------------------------------------------------

spread = full_table[full_table['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len)
spread = pd.DataFrame(spread).reset_index()

fig_5 = px.line(spread, x='Date', y='Country/Region',  color_discrete_sequence=[dth])

# -------------------------------------------------------------------------------------------

temp = full_table.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths']
temp = temp.sum().diff().reset_index()

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan

temp = temp.groupby('Date')['Confirmed'].sum().reset_index()

fig_6 = px.bar(temp, x="Date", y="Confirmed", color_discrete_sequence=['#2a7886'])
fig_6.update_layout(xaxis_rangeslider_visible=True)

# ==========================================================================================

fig = make_subplots(rows=3, cols=2, shared_xaxes=False, vertical_spacing=0.08, horizontal_spacing=0.1,
                    subplot_titles=('Confirmed cases', 'Deaths reported', 
                                    'No. of deaths to 100 Confirmed', 'No. of recovered cases to 100 Confirmed', 
                                    'No. of new countries everyday', 'No. of new cases everyday'))

fig.add_trace(fig_1['data'][0], row=1, col=1)
fig.add_trace(fig_2['data'][0], row=1, col=2)
fig.add_trace(fig_3['data'][0], row=2, col=1)
fig.add_trace(fig_4['data'][0], row=2, col=2)
fig.add_trace(fig_5['data'][0], row=3, col=1)
fig.add_trace(fig_6['data'][0], row=3, col=2)

fig.update_layout(height=1200)

In [116]:
flg = full_latest_grouped.copy('deept')
# flg.head()

# mortality rate

flg['Mortality Rate'] = round((flg['Deaths']/flg['Confirmed'])*100, 2)
temp_m = flg[flg['Confirmed']>100]
temp_m = temp_m.sort_values('Mortality Rate', ascending=False)

# load population dataset
pop_clean_data = pd.read_csv("../input/world-population-19602018/population_total_long.csv")
pop = pop_clean_data[pop_clean_data['Year']==max(pop_clean_data['Year'])]
# select only the latest data
pop = pop.loc[:, ['Country Name', 'Count']]

# substitute for missing values
cols = ['French Guiana', 'Martinique', 'Palestine' , 'Republic of Ireland', 'Saint Barthelemy', 'Taiwan', 'Vatican City']
pops = [290691, 376480, 4750000, 4920455, 7122, 23476640, 1000]
for c, p in zip(cols, pops):
    pop.loc[pop['Country Name']== c, 'Count'] = p

# replace country names 
name = {
    'Egypt, Arab Rep.': 'Egypt',
    'Hong Kong SAR, China': 'Hong Kong',
    'Iran, Islamic Rep.': 'Iran (Islamic Republic of)',
    'Macao SAR, China': 'Macau',
    'Hong Kong SAR': 'Hong Kong',
    'Russian Federation': 'Russia',
    'Slovak Republic': 'Slovakia',
    'Korea, Rep.': 'Republic of Korea',
    'United Kingdom': 'UK',
    'United States': 'US' }
pop.loc[:, 'Country Name'] = pop.loc[: 'Country Name'].replace(name)

# merge dataframes
temp = pd.merge(full_latest_grouped, pop, how='left', right_on='Country Name', left_on='Country/Region')
# print(temp[temp['Country Name'].isna()])

temp = temp[['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Count']]
temp.columns = ['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population']
    
# calculate Confirmed/Population
temp['Confirmed Per Million People'] = round(temp['Confirmed']/temp['Population']*1000000, 2)

# countries with population greater that 1 million only
temp_p = temp[temp['Population']>1000000].sort_values('Confirmed Per Million People', ascending=False).reset_index(drop=True)
# temp_p.head()

In [117]:
# Confirmed ==================================================================================================

fig_c = px.bar(flg.sort_values('Confirmed').tail(20), x="Confirmed", y="Country/Region", text='Confirmed', orientation='h')
fig_c.update_traces(marker_color='#000839', opacity=0.8, textposition='auto')
fig_c.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Deaths ===================================================================================================

fig_d = px.bar(flg.sort_values('Deaths').tail(20), x="Deaths", y="Country/Region",  text='Deaths', orientation='h')
fig_d.update_traces(marker_color='#ff1e56', opacity=0.8, textposition='auto')
fig_d.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Recovered =================================================================================================

fig_r = px.bar(flg.sort_values('Recovered').tail(20), x="Recovered", y="Country/Region", text='Recovered', orientation='h')
fig_r.update_traces(marker_color=rec, opacity=0.8, textposition='auto')
fig_r.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Active ====================================================================================================

fig_a = px.bar(flg.sort_values('Active').tail(20), x="Active", y="Country/Region", text='Active', orientation='h')
fig_a.update_traces(marker_color=act, opacity=0.8, textposition='auto')
fig_a.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Moratality ================================================================================================

fig_m = px.bar(temp_m.sort_values('Mortality Rate').tail(20), x="Mortality Rate", y="Country/Region", text='Mortality Rate', orientation='h')
fig_m.update_traces(marker_color='#00bdaa', opacity=0.8, textposition='auto')
fig_m.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Per Million People =================================================================================================

fig_p = px.bar(temp_p.sort_values('Confirmed Per Million People').tail(20), x='Confirmed Per Million People', y='Country/Region', orientation='h', text='Confirmed Per Million People')
fig_p.update_traces(marker_color='#2c003e', opacity=0.8, textposition='auto')
fig_p.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# ==========================================================================================================

fig = make_subplots(rows=3, cols=2, shared_xaxes=False, vertical_spacing=0.08, horizontal_spacing=0.1,
                    subplot_titles=("Confirmed", "Deaths", "Recovered", "Active", "Moratlity rate", "Confirmed cases per Million People"))
fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)
fig.add_trace(fig_r['data'][0], row=2, col=1)
fig.add_trace(fig_a['data'][0], row=2, col=2)
fig.add_trace(fig_m['data'][0], row=3, col=1)
fig.add_trace(fig_p['data'][0], row=3, col=2)
fig.update_layout(height=1200, title_text="Top 20")

In [119]:
epidemics = pd.DataFrame({
    'epidemic' : ['COVID-19', 'SARS', 'EBOLA', 'MERS', 'H1N1'],
    'start_year' : [2019, 2003, 2014, 2012, 2009],
    'end_year' : [2020, 2004, 2016, 2017, 2010],
    'confirmed' : [full_latest['Confirmed'].sum(), 8096, 28646, 2494, 6724149],
    'deaths' : [full_latest['Deaths'].sum(), 774, 11323, 858, 19654]
})

epidemics['mortality'] = round((epidemics['deaths']/epidemics['confirmed'])*100, 2)

epidemics.head()

,epidemic,start_year,end_year,confirmed,deaths,mortality
0,COVID-19,2019,2020,593291,27198,4.58
1,SARS,2003,2004,8096,774,9.56
2,EBOLA,2014,2016,28646,11323,39.53
3,MERS,2012,2017,2494,858,34.40
4,H1N1,2009,2010,6724149,19654,0.29


In [121]:
temp = epidemics.melt(id_vars='epidemic', value_vars=['confirmed', 'deaths', 'mortality'],
                      var_name='Case', value_name='Value')

fig = px.bar(temp, x="epidemic", y="Value", color='epidemic', text='Value', facet_col="Case",
             color_discrete_sequence = px.colors.qualitative.Bold)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_yaxes(showticklabels=False)
fig.layout.yaxis2.update(matches=None)
fig.layout.yaxis3.update(matches=None)
fig.show()

In [ ]:
temp = epidemics.melt(id_vars='epidemic', value_vars=['confirmed', 'deaths', 'mortality'],
                      var_name='Case', value_name='Value')

fig = px.bar(temp, x="epidemic", y="Value", color='epidemic', text='Value', facet_col="Case",
             color_discrete_sequence = px.colors.qualitative.Bold)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_yaxes(showticklabels=False)
fig.layout.yaxis2.update(matches=None)
fig.layout.yaxis3.update(matches=None)
fig.show()